In [1]:
import csv
import datetime
import dotenv
import glob
import json
import os
import pprint
import time

import boto3
import pandas as pd

In [2]:
print("boto3", boto3.__version__)
print("pandas", pd.__version__)

boto3 1.40.21
pandas 2.3.0


In [ ]:
dotenv.load_dotenv()
CLAUDE_MODEL_ID = os.getenv("BEDROCK_CLAUDE_MODEL_ID")
CLAUDE_CLIENT = boto3.client("bedrock-runtime", region_name="ap-northeast-1")
FUNCTION_CALLING_NAME = "hogehoge"
FUNCTION_CALLING_DESCRIPTION = """
あなたはコンタクトセンターで音声通話テキストからお客様のニーズ(VOC)を把握する業務を10年以上行っている専門家です。
与えられたテキストは過去の通話を記録したもので、CUはお客様を表し、OPはオペレータの発言になります。
"""
INPUT_DATA_PATH = "複数行1列の形で音声通話テキストが保存されているcsvファイル"
OUTPUT_FOLDER = "出力ファイルの保存先のディレクトリ"

In [ ]:
NAME_1_IN_FUNCTION_CALLING = "predict_customer_request_by_LLM"
PROMPT_1_IN_FUNCTION_CALLING = "会話はコンタクトセンターでの電話のやり取りです。このやり取りの中から、CUの要望を推測して、時系列順に箇条書きのリストにする事。"
NAME_2_IN_FUNCTION_CALLING = "judge_achieve_customer_request_by_LLM"
PROMPT_2_IN_FUNCTION_CALLING = "お客様の{a}のそれぞれの要望が満たされたかどうかを、要望毎に推測してリストにする事。".format(a=NAME_1_IN_FUNCTION_CALLING)
NAME_3_IN_FUNCTION_CALLING = "reason_of_achievement_judge_by_LLM"
PROMPT_3_IN_FUNCTION_CALLING = "{a}のそれぞれの評価になった理由を、要望毎に説明してリストにする事。".format(a=NAME_2_IN_FUNCTION_CALLING)
NAME_4_IN_FUNCTION_CALLING = "judge_customer_satisfaction_score_by_LLM"
PROMPT_4_IN_FUNCTION_CALLING = "オペレータの応対に対してのお客様の満足度を推測して数値で表すこと。不満足ならば0、満足ならば100とする。(0-100)"

In [ ]:
def structured_output_tool():
    return {"toolSpec": {"name": FUNCTION_CALLING_NAME,
                         "description": FUNCTION_CALLING_DESCRIPTION,
                         "inputSchema": {"json": {"type": "object",
                                                  "properties": {NAME_1_IN_FUNCTION_CALLING: {"type": "array",
                                                                                              "items": {"type": "string"},
                                                                                              "description": PROMPT_1_IN_FUNCTION_CALLING},
                                                                 NAME_2_IN_FUNCTION_CALLING: {"type": "array",
                                                                                              "items": {"type": "string"},
                                                                                              "description": PROMPT_2_IN_FUNCTION_CALLING,
                                                                                              "enum": ["True", "False"]},
                                                                 NAME_3_IN_FUNCTION_CALLING: {"type": "array",
                                                                                              "items": {"type": "string"},
                                                                                              "description": PROMPT_3_IN_FUNCTION_CALLING},
                                                                 NAME_4_IN_FUNCTION_CALLING: {"type": "number",
                                                                                              "description": PROMPT_4_IN_FUNCTION_CALLING}
                                                                },
                                                  "required": [NAME_1_IN_FUNCTION_CALLING,
                                                               NAME_2_IN_FUNCTION_CALLING,
                                                               NAME_3_IN_FUNCTION_CALLING,
                                                               NAME_4_IN_FUNCTION_CALLING]
                                                 }
                                        }
                        }
           }


def process_claude_with_function_calling(call_text_and_speaker):
    prompt = """
    <text>
    {a}
    </text>
    """.format(a=call_text_and_speaker)
    claude_input_with_prompt = [{"role": "user",
                                 "content": [{"text": prompt}]
                                }]
    claude_response = CLAUDE_CLIENT.converse(modelId=CLAUDE_MODEL_ID,
                                             messages=claude_input_with_prompt,
                                             inferenceConfig={"temperature": 0.2},
                                             toolConfig={"tools": [structured_output_tool()],
                                                         "toolChoice": {"tool": {"name": FUNCTION_CALLING_NAME}}
                                                        })
    result = claude_response["output"]["message"]["content"][0]["toolUse"]["input"]
    return result

In [ ]:
def read_csv_file_and_inference_using_claude(csv_file_path):
    df = pd.DataFrame()
    with open(file=csv_file_path, mode="r", encoding="utf-8") as csvfile:
        csvfile_data_reader = csv.reader(csvfile)
        next(csvfile_data_reader, None)  # 1行目のヘッダーをスキップする
        print(datetime.datetime.now())
        conversation_id = 0  # 便宜上の会話ID
        for row in csvfile_data_reader:
            if row:
                conversation_id += 1
                predict_result = process_claude_with_function_calling(call_text_and_speaker=row[0])  # 変数rowがlist型だったので[0]でstringを取得
                dict_for_dataframe = {"conversation_ID": conversation_id,  # 数値
                                      "comment": row[0],  # 文字列
                                      "customer_request": predict_result["predict_customer_request_by_LLM"],  # リスト
                                      "judge_achievement": predict_result["judge_achieve_customer_request_by_LLM"],  # リスト
                                      "reason_of_achievement": predict_result["reason_of_achievement_judge_by_LLM"],  # リスト
                                      "satisfaction_score": predict_result["judge_customer_satisfaction_score_by_LLM"]}  # 数値
                try:
                    temp_df = pd.DataFrame(data=dict_for_dataframe)
                    temp_df = temp_df.explode(["customer_request", "judge_achievement", "reason_of_achievement"])
                except Exception as e:
                    print(e)
                    continue
                df = pd.concat(objs=[df, temp_df], axis=0)
                time.sleep(1)  # bedrockのapiリクエスト制限を防ぐための待ち
            else:
                continue
        print(datetime.datetime.now())
    return df

In [ ]:
df = read_csv_file_and_inference_using_claude(csv_file_path=INPUT_DATA_PATH)
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)
df.to_csv(path_or_buf="{a}/hogehoge_result.csv".format(a=OUTPUT_FOLDER),
          index=False,
          encoding="utf-8")